In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, operator
%matplotlib inline

###Class Distribution

#### Calculate fraction of documents in each class

In [4]:
#Training label
train_label = open('/home/sadat/Downloads/HW2_210/20news-bydate/matlab/train.label')

#pi is the fraction of each class
pi = {}

#Set a class index for each document as key
for i in range(1,21):
    pi[i] = 0
    
#Extract values from training labels
lines = train_label.readlines()

#Get total number of documents
total = len(lines)

#Count the occurence of each class
for line in lines:
    val = int(line.split()[0])
    pi[val] += 1

#Divide the count of each class by total documents 
for key in pi:
    pi[key] /= total
    
print("Probability of each class:")
print("\n".join("{}: {}".format(k, v) for k, v in pi.items()))

Probability of each class:
1: 0.04259472890229834
2: 0.05155736977549028
3: 0.05075871860857219
4: 0.05208980388676901
5: 0.051024935664211554
6: 0.052533498979501284
7: 0.051646108794036735
8: 0.052533498979501284
9: 0.052888455053687104
10: 0.0527109770165942
11: 0.05306593309078002
12: 0.0527109770165942
13: 0.05244475996095483
14: 0.0527109770165942
15: 0.052622237998047744
16: 0.05315467210932647
17: 0.04836276510781791
18: 0.05004880646020055
19: 0.04117490460555506
20: 0.033365870973467035


In [5]:
#Check if sum of the probabilities is 1
np.sum(list(pi.values()))

1.0

###Probability Distribution over V

####Dataframe

In [6]:
#Training data
train_data = open('/home/sadat/Downloads/HW2_210/20news-bydate/matlab/train.data')
df = pd.read_csv(train_data, delimiter=' ', names=['docIdx', 'wordIdx', 'count'])

#Training label
label = []
train_label = open('/home/sadat/Downloads/HW2_210/20news-bydate/matlab/train.label')
lines = train_label.readlines()
for line in lines:
    label.append(int(line.split()[0]))

#Increase label length to match docIdx
docIdx = df['docIdx'].values
i = 0
new_label = []
for index in range(len(docIdx)-1):
    new_label.append(label[i])
    if docIdx[index] != docIdx[index+1]:
        i += 1
new_label.append(label[i]) #for-loop ignores last value

#Add label column
df['label'] = new_label

df.head()

,docIdx,wordIdx,count,label
0,1,1,4,1
1,1,2,2,1
2,1,3,10,1
3,1,4,4,1
4,1,5,2,1


####Probability of each word per class

For calculating our probability, we will find the average of each word for a given class.

For class j and word i, the average is given by:

$$P(i|j) = \frac{count_{ij}}{count_j}$$


However, since some words will have 0 counts, we will perform a Laplace Smoothing:



$$ P(i|j) = log(\frac{count_{ij}+1}{count_j+|V|})$$

In [7]:
#Calculate probability of each word based on class
pb_ij = df.groupby(['label','wordIdx'])
pb_j = df.groupby(['label'])
Pr =  (pb_ij['count'].sum() + 1) / (pb_j['count'].sum() + 16689)    

#Unstack series
Pr = Pr.unstack()

#Replace NaN or columns with 0 as word count with 1/(|V|+1)
Pr = Pr.fillna(1/16689)
Pr

wordIdx,1,2,3,4,5,6,7,8,9,10,...,53966,53967,53968,53969,53970,53971,53972,53973,53974,53975
label,,,,,,,,,,,,,,,,,,,,,
1,0.000085,0.000387,0.001668,0.000060,0.000502,0.000254,0.000042,0.000012,0.000211,0.000852,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
2,0.000480,0.000472,0.000060,0.000142,0.000118,0.000464,0.000087,0.000055,0.001362,0.000031,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
3,0.000112,0.000651,0.000060,0.000168,0.000205,0.000326,0.000028,0.000028,0.001349,0.000060,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
4,0.000078,0.000276,0.000060,0.000060,0.000095,0.000423,0.000026,0.000017,0.000423,0.000060,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
5,0.000068,0.000330,0.000060,0.000019,0.000019,0.000467,0.000019,0.000060,0.000467,0.000060,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
6,0.000283,0.001315,0.000060,0.000472,0.000094,0.000313,0.000130,0.000024,0.001404,0.000060,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
7,0.000060,0.000373,0.000060,0.000039,0.000039,0.000424,0.000060,0.000060,0.000373,0.000051,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
8,0.000076,0.000421,0.000060,0.000060,0.000107,0.000665,0.000061,0.000031,0.000145,0.000060,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
9,0.000126,0.000570,0.000060,0.000042,0.000042,0.000704,0.000034,0.000017,0.000042,0.000060,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060


###Multinomial Naive Bayes Classifier

Combining probability distribution of P with fraction of documents belonging to each class

$$Pr(i|j)=\pi_j * P_{ij} $$

In [8]:
for i in range(1,21):
    Pr[:][i]=Pr[:][i]*pi[i]
    
Pr

wordIdx,1,2,3,4,5,6,7,8,9,10,...,53966,53967,53968,53969,53970,53971,53972,53973,53974,53975
label,,,,,,,,,,,,,,,,,,,,,
1,3.603158e-06,0.000020,0.000085,0.000003,2.558939e-05,0.000013,2.184414e-06,6.348421e-07,0.000011,4.490757e-05,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
2,2.045132e-05,0.000024,0.000003,0.000007,6.024338e-06,0.000024,4.471630e-06,2.894476e-06,0.000072,1.659574e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
3,4.756707e-06,0.000034,0.000003,0.000009,1.044659e-05,0.000017,1.441877e-06,1.466651e-06,0.000071,3.158426e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
4,3.309471e-06,0.000014,0.000003,0.000003,4.845464e-06,0.000022,1.337578e-06,9.070402e-07,0.000022,3.158426e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
5,2.898192e-06,0.000017,0.000003,0.000001,9.919407e-07,0.000025,1.004017e-06,3.147792e-06,0.000025,3.158426e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
6,1.205973e-05,0.000068,0.000003,0.000025,4.815519e-06,0.000016,6.701946e-06,1.239473e-06,0.000074,3.158426e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
7,2.552264e-06,0.000019,0.000003,0.000002,1.967973e-06,0.000022,3.094620e-06,3.147792e-06,0.000020,2.710668e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
8,3.256702e-06,0.000022,0.000003,0.000003,5.461760e-06,0.000035,3.159001e-06,1.606640e-06,0.000008,3.158426e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
9,5.354684e-06,0.000029,0.000003,0.000002,2.138155e-06,0.000037,1.731348e-06,8.805481e-07,0.000002,3.158426e-06,...,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060


In [9]:
#Convert to dictionary for greater speed
Pr_dict = Pr.to_dict()

#### Generating function

In [34]:
def MNB(df):
    #Avoid damaging the original df
    new_df = df
    
    #Creating a probability row for each class
    for i in range(1,21):
        new_df[i] = new_df['wordIdx'].apply(lambda x: Pr_dict[x][i]) * new_df['count'].apply(lambda x: np.log(x+1))
    new_df['count'].drop()
    classes = [int(i) for i in range(1,21)]
    predict_prob = new_df.reset_index()
    predict_prob = predict_prob.set_index(['docIdx', 'wordIdx', 'index'])
    predict_prob = predict_prob.groupby(level=[0,1,2]).sum().groupby(level=[0,1]).cumsum()
    #predict_class = preict_prob.idxmax(axis=1)
    return predict_prob.values

In [35]:
MNB(df)

KeyError: 'Index'

#Dear Michael, everything after this is random shit

In [47]:
#df = df.reset_index()
df.columns

Index(['level_0',   'index',  'docIdx', 'wordIdx',   'count',   'label',
               1,         2,         3,         4,         5,         6,
               7,         8,         9,        10,        11,        12,
              13,        14,        15,        16,        17,        18,
              19,        20],
      dtype='object')

In [59]:
df = df.reset_index
df.set_index['docIdx','wordIdx'].groupby(level=0).cumsum()

AttributeError: 'function' object has no attribute 'set_index'

In [60]:
df

<bound method DataFrame.reset_index of                              level_0    count     label            1  \
docIdx wordIdx index                                                   
1      1       0        0.000000e+00        4         1     0.000006   
       2       1        1.000000e+00        6         2     0.000028   
       3       2        3.000000e+00       16         3     0.000231   
       4       3        6.000000e+00       20         4     0.000236   
       5       4        1.000000e+01       22         5     0.000264   
       6       5        1.500000e+01       23         6     0.000273   
       7       6        2.100000e+01       24         7     0.000275   
       8       7        2.800000e+01       25         8     0.000275   
       9       8        3.600000e+01       28         9     0.000291   
       10      9        4.500000e+01       37        10     0.000394   
       11      10       5.500000e+01       39        11     0.000395   
       12      11       6